In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as DT
import warnings as wr
import xgboost as xgb
import matplotlib.pyplot as plt
import sys

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

%matplotlib inline
wr.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)
pd.set_option('display.max_columns', 1000)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
################################################################
## INSTALLS.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
installs = pd.read_csv('csv/installs.csv')
## TRANSFORMACIÓN DE TIPOS PARA EL CSV (INSTALLS)
installs['kind'] = installs['kind'].astype('category')
installs['user_agent'] = installs['user_agent'].astype('category')
installs['session_user_agent'] = installs['session_user_agent'].astype('category')
installs['created'] = pd.to_datetime(installs['created'], infer_datetime_format=True)
installs['trans_id'] = installs['trans_id'].astype('category')
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
installs['mes'] = installs['created'].dt.month
installs['dia'] = installs['created'].dt.day
installs['hora'] = installs['created'].dt.hour
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
installs['hora_madrugada'] = 0
installs.loc[((installs.hora > -1) & (installs.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
installs['hora_maniana'] = 0
installs.loc[((installs.hora > 6) & (installs.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
installs['hora_almuerzo'] = 0
installs.loc[((installs.hora > 11) & (installs.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
installs['hora_tarde'] = 0
installs.loc[((installs.hora > 13) & (installs.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
installs['hora_noche'] = 0
installs.loc[((installs.hora > 18) & (installs.hora < 24)), 'hora_noche'] = 1
installs['cantidad'] = 1
################################################################
################################################################

In [3]:
##################### fillna
installs['click_hash'].fillna(0, inplace = True)
installs['device_brand'].fillna(0, inplace = True)
installs['device_model'].fillna(0, inplace = True)
installs['device_language'].fillna(0, inplace = True)
##################### created event
del installs['event_uuid']
#del installs['created']
##################### wifi
installs['wifi_value'] = 2
installs.loc[installs.wifi == False, 'wifi_value'] = 0
installs.loc[installs.wifi == True, 'wifi_value'] = 1
del installs['wifi']
##################### attributed
installs['attributed_value'] = 2
installs.loc[installs.attributed == False, 'attributed_value'] = 0
installs.loc[installs.attributed == True, 'attributed_value'] = 1
del installs['attributed']
##################### implicit
installs['implicit_value'] = 2
installs.loc[installs.implicit == False, 'implicit_value'] = 0
installs.loc[installs.implicit == True, 'implicit_value'] = 1
del installs['implicit']
##################### click_hash
installs['click_hash'] = installs['click_hash'].astype('category')
##################### session_user_agent
installs['session_user_agent_value'] = installs['session_user_agent'].cat.codes
del installs['session_user_agent']
installs.loc[installs.session_user_agent_value == -1, 'session_user_agent_value'] = 0
##################### click_hash
installs['click_hash_value'] = installs['click_hash'].cat.codes
installs.loc[installs.click_hash_value == -1, 'click_hash_value'] = 0
del installs['click_hash']
##################### user_agent
installs['user_agent_value'] = installs['user_agent'].cat.codes
installs.loc[installs.user_agent_value == -1, 'user_agent_value'] = 0
del installs['user_agent']
##################### kind
installs['kind_value'] = installs['kind'].cat.codes
installs.loc[installs.kind_value == -1, 'kind_value'] = 0
del installs['kind']
##################### trans_id
installs['trans_id_value'] = installs['trans_id'].cat.codes
installs.loc[installs.trans_id_value == -1, 'trans_id_value'] = 0
del installs['trans_id']
##################### pasamos columnas a string.
installs['ref_type'] = installs['ref_type'].astype(str)
installs['wifi_value'] = installs['wifi_value'].astype(str)
installs['implicit_value'] = installs['implicit_value'].astype(str)

installs.loc[installs.ref_type == '1891515180541284343', 'ref_type'] = 'ref_type_1'
installs.loc[installs.ref_type == '1494519392962156891', 'ref_type'] = 'ref_type_2'

installs.loc[installs.wifi_value == '0', 'wifi_value'] = 'wifi_value_0'
installs.loc[installs.wifi_value == '1', 'wifi_value'] = 'wifi_value_1'
installs.loc[installs.wifi_value == '2', 'wifi_value'] = 'wifi_value_2'

installs.loc[installs.implicit_value == '0', 'implicit_value'] = 'implicit_value_0'
installs.loc[installs.implicit_value == '1', 'implicit_value'] = 'implicit_value_1'

dummies = pd.get_dummies(installs['ref_type'], drop_first=False)
installs = pd.concat([installs, dummies], axis=1)
dummies = pd.get_dummies(installs['wifi_value'], drop_first=False)
installs = pd.concat([installs, dummies], axis=1)
dummies = pd.get_dummies(installs['implicit_value'], drop_first=False)
installs = pd.concat([installs, dummies], axis=1)

del installs['ref_type']
del installs['wifi_value']
del installs['implicit_value']

In [4]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481511 entries, 0 to 481510
Data columns (total 30 columns):
created                     481511 non-null datetime64[ns]
application_id              481511 non-null int64
ref_hash                    481511 non-null int64
device_countrycode          481511 non-null int64
device_brand                481511 non-null float64
device_model                481511 non-null float64
ip_address                  481511 non-null int64
device_language             481511 non-null float64
mes                         481511 non-null int64
dia                         481511 non-null int64
hora                        481511 non-null int64
hora_madrugada              481511 non-null int64
hora_maniana                481511 non-null int64
hora_almuerzo               481511 non-null int64
hora_tarde                  481511 non-null int64
hora_noche                  481511 non-null int64
cantidad                    481511 non-null int64
attributed_value        

In [5]:
################################################################
## Ventana 1: Del 18 al 20
## Ventana 2: Del 19 al 21
## Ventana 3: Del 20 al 22
## Ventana 4: Del 21 al 23
## Ventana 5: Del 22 al 24
################################################################
installs_Ventana1 = installs[(installs['dia'] >= 21) & (installs['dia'] <= 23)]
installs_Ventana2 = installs[(installs['dia'] >= 24) & (installs['dia'] <= 26)]
#installs_Ventana3 = installs[(installs['dia'] >= 20) & (installs['dia'] <= 22)]
#installs_Ventana4 = installs[(installs['dia'] >= 21) & (installs['dia'] <= 23)]
#installs_Ventana5 = installs[(installs['dia'] >= 22) & (installs['dia'] <= 24)]
################################################################

In [6]:
################################################################
installs['ref_hash'] = installs['ref_hash'].astype(str)
installs['ref_hash'] = installs['ref_hash'] + '_sc'
installs_Ventana1['ref_hash'] = installs_Ventana1['ref_hash'].astype(str)
installs_Ventana1['ref_hash'] = installs_Ventana1['ref_hash'] + '_sc'
installs_Ventana2['ref_hash'] = installs_Ventana2['ref_hash'].astype(str)
installs_Ventana2['ref_hash'] = installs_Ventana2['ref_hash'] + '_sc'
################################################################
installs_cantidad_0 = installs[['ref_hash', 'cantidad', 'ref_type_1', 'ref_type_2', 'wifi_value_0', 'wifi_value_1', 'wifi_value_2', 'implicit_value_0', 'implicit_value_1']].copy()
installs_cantidad_1 = installs_Ventana1[['ref_hash', 'cantidad', 'ref_type_1', 'ref_type_2', 'wifi_value_0', 'wifi_value_1', 'wifi_value_2', 'implicit_value_0', 'implicit_value_1']].copy()
installs_cantidad_2 = installs_Ventana2[['ref_hash', 'cantidad', 'ref_type_1', 'ref_type_2', 'wifi_value_0', 'wifi_value_1', 'wifi_value_2', 'implicit_value_0', 'implicit_value_1']].copy()
inst_cant_0 = installs_cantidad_0.groupby('ref_hash').sum()
inst_cant_1 = installs_cantidad_1.groupby('ref_hash').sum()
inst_cant_2 = installs_cantidad_2.groupby('ref_hash').sum()
################################################################
installs.drop(['cantidad', 'ref_type_1', 'ref_type_2', 'wifi_value_0', 'wifi_value_1', 'wifi_value_2', 'implicit_value_0', 'implicit_value_1'], axis=1, inplace=True)
installs_Ventana1.drop(['cantidad', 'ref_type_1', 'ref_type_2', 'wifi_value_0', 'wifi_value_1', 'wifi_value_2', 'implicit_value_0', 'implicit_value_1'], axis=1, inplace=True)
installs_Ventana2.drop(['cantidad', 'ref_type_1', 'ref_type_2', 'wifi_value_0', 'wifi_value_1', 'wifi_value_2', 'implicit_value_0', 'implicit_value_1'], axis=1, inplace=True)
inst_1 = installs_Ventana1.groupby('ref_hash').min()
inst_2 = installs_Ventana2.groupby('ref_hash').min()
################################################################

In [7]:
inst_comb_1 = inst_1.merge(inst_cant_0, on=('ref_hash'))

In [8]:
inst_comb_1.sort_values(by='cantidad', ascending=False).head()

,created,application_id,device_countrycode,device_brand,device_model,ip_address,device_language,mes,dia,hora,hora_madrugada,hora_maniana,hora_almuerzo,hora_tarde,hora_noche,attributed_value,session_user_agent_value,click_hash_value,user_agent_value,kind_value,trans_id_value,cantidad,ref_type_1,ref_type_2,wifi_value_0,wifi_value_1,wifi_value_2,implicit_value_0,implicit_value_1
ref_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5446085605337844584_sc,2019-04-21 22:10:02.158,21,6287817205707153877,0.000000e+00,0.000000e+00,1753742327176262608,0.000000e+00,4,21,13,0,0,0,0,0,0,0,0,0,0,0,14,14,0,3,0,11,10,4
906973248467925335_sc,2019-04-21 23:36:16.992,55,6287817205707153877,0.000000e+00,0.000000e+00,534436939163621182,0.000000e+00,4,21,13,0,0,0,0,0,0,4528,0,0,0,0,14,14,0,0,8,6,8,6
6337720443035762926_sc,2019-04-22 05:49:32.837,75,6287817205707153877,6.115026e+18,4.153661e+18,5475639131460325130,6.977049e+18,4,22,5,0,0,0,0,0,0,4530,0,6294,96,0,11,11,0,1,6,4,7,4
2328515224695623403_sc,2019-04-21 16:37:39.592,49,6287817205707153877,0.000000e+00,6.794880e+18,2667830573880102966,8.441417e+18,4,21,16,0,0,0,0,0,0,4530,0,98,0,0,10,0,10,0,7,3,6,4
5472947665337192726_sc,2019-04-23 01:16:24.557,61,6287817205707153877,0.000000e+00,0.000000e+00,1255064889345628723,0.000000e+00,4,23,1,0,0,0,0,0,0,4530,0,0,0,0,10,10,0,3,4,3,7,3


In [9]:
################################################################
## AUCTIONS.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
auctions = pd.read_csv('csv/auctions.csv',nrows=30000000)
## TRANSFORMACIÓN DE TIPOS PARA EL CSV (AUCTIONS)
auctions['date'] = pd.to_datetime(auctions['date'], infer_datetime_format=True)
## DIFERENCIAMOS EN TRES COLUMNAS DIFERENTES EL DIA, MES Y AÑO.
auctions['mes'] = auctions['date'].dt.month
auctions['dia'] = auctions['date'].dt.day
auctions['hora'] = auctions['date'].dt.hour
# ARMAMOS UNA LÓGICA PARA SEGMENTAR LAS FRANJAS HORARIAS.
# MADRUGADA de 00 a 06
auctions['hora_madrugada'] = 0
auctions.loc[((auctions.hora > -1) & (auctions.hora < 7)), 'hora_madrugada'] = 1
# MAÑANA de 07 a 11
auctions['hora_maniana'] = 0
auctions.loc[((auctions.hora > 6) & (auctions.hora < 12)), 'hora_maniana'] = 1
# ALMUERZO de 12 a 13
auctions['hora_almuerzo'] = 0
auctions.loc[((auctions.hora > 11) & (auctions.hora < 14)), 'hora_almuerzo'] = 1
# TARDE de 14 a 18
auctions['hora_tarde'] = 0
auctions.loc[((auctions.hora > 13) & (auctions.hora < 19)), 'hora_tarde'] = 1
# NOCHE de 19 a 23
auctions['hora_noche'] = 0
auctions.loc[((auctions.hora > 18) & (auctions.hora < 24)), 'hora_noche'] = 1
# RENOMBRAMOS LA COLUMNA PARA LOS JOINS.
auctions=auctions.rename(columns = {'device_id':'ref_hash'})
auctions=auctions.rename(columns = {'date':'created'})
#auctions['cantidad'] = 1
################################################################
################################################################

In [10]:
################################################################
## Ventana 1: Del 18 al 20
## Ventana 2: Del 19 al 21
## Ventana 3: Del 20 al 22
## Ventana 4: Del 21 al 23
## Ventana 5: Del 22 al 24
################################################################
auctions_Ventana1 = auctions[(auctions['dia'] >= 21) & (auctions['dia'] <= 23)]
auctions_Ventana2 = auctions[(auctions['dia'] >= 24) & (auctions['dia'] <= 26)]
#auctions_Ventana3 = auctions[(auctions['dia'] >= 20) & (auctions['dia'] <= 22)]
#auctions_Ventana4 = auctions[(auctions['dia'] >= 20) & (auctions['dia'] <= 23)]
#auctions_Ventana5 = auctions[(auctions['dia'] >= 23) & (auctions['dia'] <= 27)]
################################################################

In [11]:
auctions['ref_hash'] = auctions['ref_hash'].astype(str)
auctions['ref_hash'] = auctions['ref_hash'] + '_st'
auctions_Ventana1['ref_hash'] = auctions_Ventana1['ref_hash'].astype(str)
auctions_Ventana1['ref_hash'] = auctions_Ventana1['ref_hash'] + '_st'
auctions_Ventana2['ref_hash'] = auctions_Ventana2['ref_hash'].astype(str)
auctions_Ventana2['ref_hash'] = auctions_Ventana2['ref_hash'] + '_st'

In [12]:
################################################################
## TARGET.
################################################################
## OBTENEMOS TODA LA INFORMACIÓN DE LOS DIFERENTES CSV.
targets = pd.read_csv('csv/target.csv')
targets_sc = targets[targets['ref_hash'].str.contains('_sc')]
targets_st = targets[targets['ref_hash'].str.contains('_st')]
################################################################
################################################################

## =================================================================
## JUNTAMOS LOS DATAFRAMES Y EMPEZAMOS A ENTRENAR (SC = INSTALLS).
## =================================================================

In [13]:
targets['ref_hash'] = targets['ref_hash'].astype(str)
targets_st['ref_hash'] = targets_st['ref_hash'].astype(str)
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(str)

In [14]:
inst_1 = installs_Ventana1.groupby('ref_hash').min()
inst_2 = installs_Ventana2.groupby('ref_hash').min()
inst_1['obj'] = 0

In [15]:
inst_comb = inst_1.merge(inst_2, on=('ref_hash'), suffixes=('_lefto', '_raito'))
inst_comb['obj'] = (inst_comb['created_raito'] - inst_comb['created_lefto']).dt.total_seconds()

In [16]:
inst_comb = inst_comb.iloc[:, 0:25]
inst_comb['obj'] = inst_comb['obj'].astype(int)

In [17]:
inst_comb.dtypes

created_lefto                     datetime64[ns]
application_id_lefto                       int64
device_countrycode_lefto                   int64
device_brand_lefto                       float64
device_model_lefto                       float64
ip_address_lefto                           int64
device_language_lefto                    float64
mes_lefto                                  int64
dia_lefto                                  int64
hora_lefto                                 int64
hora_madrugada_lefto                       int64
hora_maniana_lefto                         int64
hora_almuerzo_lefto                        int64
hora_tarde_lefto                           int64
hora_noche_lefto                           int64
attributed_value_lefto                     int64
session_user_agent_value_lefto             int16
click_hash_value_lefto                     int16
user_agent_value_lefto                     int16
kind_value_lefto                           int16
trans_id_value_lefto

In [18]:
columnas_relevantes_inst = list(inst_comb.select_dtypes(include=['int16','int32','int64','float64','uint8']).columns)
inst_comb = inst_comb.loc[:, inst_comb.columns.isin(columnas_relevantes_inst)]

In [19]:
inst_comb = inst_comb.iloc[:, 7:19]
columnas_relevantes_inst = list(inst_2.select_dtypes(include=['int16','int32','int64','float64','uint8']).columns)
inst_2 = inst_2.loc[:, inst_2.columns.isin(columnas_relevantes_inst)]
inst_ventana2 = inst_2.iloc[:, 7:19]
target_result_sc = pd.merge(targets_sc, inst_ventana2, how='left', left_on='ref_hash', right_on='ref_hash')

In [20]:
X_inst, y_inst = inst_comb.iloc[:,:-1],inst_comb.iloc[:,-1]
data_dmatrix_inst = xgb.DMatrix(data=X_inst,label=y_inst)
X_train_inst, X_test_inst, y_train_inst, y_test_inst = train_test_split(X_inst, y_inst, test_size=0.2, random_state=123)

In [21]:
xg_reg_inst = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, 
              learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10, n_jobs = 0)

In [22]:
xg_reg_inst.fit(X_train_inst,y_train_inst)

[22:42:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=0, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [23]:
inst_ventana2.columns = [str(col) + '_lefto' for col in inst_ventana2.columns]
inst_ventana2['obj'] = 0
X_target_inst, y_target_inst = inst_ventana2.iloc[:,:-1],inst_ventana2.iloc[:,-1]
X_target_inst.drop(['kind_value_lefto'],axis=1,inplace=True)

In [24]:
preds_sc = xg_reg_inst.predict(X_target_inst)

In [25]:
## =================================================================================================
## ARMAMOS EN BASE A LA PREDICCIÓN QUE TENEMOS UN CSV PARA SUBIR A KAGGLE CON EL FORMATO INDICADO.
## =================================================================================================
inst_ventana2 = inst_ventana2.reset_index()
submission_sc = pd.DataFrame({ 'resultado': preds_sc, 'ref_hash': inst_ventana2['ref_hash'] })
final_sc = pd.merge(targets_sc, submission_sc, how='left', left_on='ref_hash', right_on='ref_hash')
final_sc['obj'] = final_sc['resultado']
del final_sc['resultado']
final_sc.fillna(0, inplace = True)

In [26]:
final_sc.head(10)

,ref_hash,obj
0,1000169251625791246_sc,0.000000
1,1000395625957344683_sc,0.000000
2,1003027494996471685_sc,0.000000
3,1006670001679961544_sc,0.000000
4,1007573308966476713_sc,0.000000
5,1010070503877148763_sc,8.010000
6,1010265377387765028_sc,0.000000
7,1010531372912327058_sc,0.000000
8,1011610998357271358_sc,0.000000
9,1013543838965040946_sc,5.641841


## =================================================================
## JUNTAMOS LOS DATAFRAMES Y EMPEZAMOS A ENTRENAR (ST = AUCTIONS).
## =================================================================

In [27]:
targets['ref_hash'] = targets['ref_hash'].astype(str)
targets_st['ref_hash'] = targets_st['ref_hash'].astype(str)
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(str)

In [28]:
auct_1 = auctions_Ventana1.groupby('ref_hash').min()
auct_2 = auctions_Ventana2.groupby('ref_hash').min()
auct_1['obj'] = 0

In [29]:
auct_comb = auct_1.merge(auct_2, on=('ref_hash'), suffixes=('_lefto', '_raito'))
auct_comb['obj'] = (auct_comb['created_raito'] - auct_comb['created_lefto']).dt.total_seconds()

In [30]:
auct_comb = auct_comb.iloc[:, 0:12]
auct_comb['obj'] = auct_comb['obj'].astype(int)
columnas_relevantes_auct = list(auct_comb.select_dtypes(include=['int16','int32','int64','float64','uint8']).columns)

In [31]:
auct_comb = auct_comb.loc[:, auct_comb.columns.isin(columnas_relevantes_auct)]
auct_comb = auct_comb.iloc[:, 2:12]
columnas_relevantes_auct = list(auct_2.select_dtypes(include=['int16','int32','int64','float64','uint8']).columns)
auct_2 = auct_2.loc[:, auct_2.columns.isin(columnas_relevantes_auct)]
auct_ventana2 = auct_2.iloc[:, 2:12]
target_result_st = pd.merge(targets_st, auct_ventana2, how='left', left_on='ref_hash', right_on='ref_hash')

In [32]:
X_auct, y_auct = auct_comb.iloc[:,:-1],auct_comb.iloc[:,-1]
data_dmatrix_auct = xgb.DMatrix(data=X_auct,label=y_auct)
X_train_auct, X_test_auct, y_train_auct, y_test_auct = train_test_split(X_auct, y_auct, test_size=0.2, random_state=123)

In [33]:
xg_reg_auct = xgb.XGBRegressor(objective = 'reg:linear', colsample_bytree = 0.3, 
         learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)

In [34]:
xg_reg_auct.fit(X_train_auct,y_train_auct)

[22:43:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [35]:
auct_ventana2.columns = [str(col) + '_lefto' for col in auct_ventana2.columns]
auct_ventana2['obj'] = 0
X_target_auct, y_target_auct = auct_ventana2.iloc[:,:-1],auct_ventana2.iloc[:,-1]
preds_st = xg_reg_auct.predict(X_target_auct)

In [36]:
## =================================================================================================
## ARMAMOS EN BASE A LA PREDICCIÓN QUE TENEMOS UN CSV PARA SUBIR A KAGGLE CON EL FORMATO INDICADO.
## =================================================================================================
auct_ventana2 = auct_ventana2.reset_index()
submission_st = pd.DataFrame({ 'resultado': preds_st, 'ref_hash': auct_ventana2['ref_hash'] })
final_st = pd.merge(targets_st, submission_st, how='left', left_on='ref_hash', right_on='ref_hash')
final_st['obj'] = final_st['resultado']
del final_st['resultado']
final_st.fillna(0, inplace = True)

In [37]:
final_st.head(10)

,ref_hash,obj
0,1000169251625791246_st,126824.390625
1,1000395625957344683_st,126824.390625
2,1003027494996471685_st,126824.390625
3,1006670001679961544_st,111925.031250
4,1007573308966476713_st,126824.390625
5,1010070503877148763_st,0.000000
6,1010265377387765028_st,0.000000
7,1010531372912327058_st,114056.406250
8,1011610998357271358_st,127625.437500
9,1013543838965040946_st,127625.437500


### ======================================================================
### GENERAMOS EL CSV PARA SUBIR A LA COMPETENCIA.
### ======================================================================

In [38]:
frames = [final_sc, final_st]
final = pd.concat(frames)
final.to_csv("submission_grupo34_003.csv", index=False)

### ======================================================================
### ======================================================================
### ======================================================================